In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models
import pandas as pd

In [13]:
import pandas as pd

# List of CSV file paths
file_paths = [
    'Collected Dataset/fist_closed.csv',
    'Collected Dataset/hand_open.csv',
    'Collected Dataset/left_click.csv',
    'Collected Dataset/move_mouse.csv',
    'Collected Dataset/move_mouse_2.csv',
    'Collected Dataset/pinch_action.csv',
    'Collected Dataset/press_and_hold_left.csv',
    'Collected Dataset/press_and_hold_left_2.csv',
    'Collected Dataset/right_click.csv',
    'Collected Dataset/double_click.csv',
    'Collected Dataset/scroll.csv'
]

# Read each CSV file into a list of DataFrames
csv_data = [pd.read_csv(file) for file in file_paths]

# Concatenate all DataFrames into a single DataFrame
data = pd.concat(csv_data, ignore_index=True)

# Shuffle the DataFrame
data = data.sample(frac=1).reset_index(drop=True)

In [14]:
string_to_numeric = {'fist_closed': 0, 'hand_open': 0, 'left_click': 1, 'right_click' : 2, 'move_mouse': 3, 'move_mouse_2' : 3, 'press_and_hold_left': 4, 'press_and_hold_left_2' : 4, 'pinch_action' : 5, 'scroll': 6 , 'double_click' : 7}

X = data.iloc[:, :-1]
y = data.iloc[:, -1]

y.replace(string_to_numeric, inplace=True)

X.sample(3)

C:\Users\ramez\AppData\Local\Temp\ipykernel_24700\1359510969.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y.replace(string_to_numeric, inplace=True)


,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,62
560,1.000000,0.0,0.926795,0.913269,0.286309,1.000000,0.733635,0.586815,0.875776,0.676401,...,0.088141,0.047374,0.070299,0.000000,0.179888,0.031063,0.151181,0.337541,0.041447,0.275965
48643,0.273299,1.0,1.000000,0.078540,0.951744,0.705572,0.000000,0.856786,0.479419,0.258486,...,0.576177,1.000000,0.870755,0.373798,0.852392,0.968467,0.381812,0.739051,0.925014,0.420969
24946,1.000000,1.0,1.000000,0.713179,0.996111,0.686413,0.432929,0.885976,0.505472,0.195773,...,0.711936,0.827986,0.295588,0.549673,0.835947,0.172523,0.514956,0.831967,0.069795,0.530067


In [15]:
num_states = len(y.unique())

X.shape, y.shape
num_states

8

In [16]:
from sklearn.model_selection import train_test_split

# Assuming you have your data stored in a numpy array called 'landmarks_data' and labels in 'labels'
# landmarks_data.shape = (num_samples, num_landmarks, num_dimensions)
# labels.shape = (num_samples,)

# First, split your data into training and testing/validation sets
X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Then, split the testing/validation set into separate testing and validation sets
X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=42)

# Print the shapes of the resulting datasets
print("Training data shape:", X_train.shape)
print("Testing data shape:", X_test.shape)
print("Validation data shape:", X_val.shape)

# You can then use X_train, y_train for training, X_test, y_test for testing, and X_val, y_val for validation

Training data shape: (42118, 63)
Testing data shape: (5265, 63)
Validation data shape: (5265, 63)


In [17]:
# Define your TensorFlow model
model = models.Sequential([
    layers.Input(shape=(63,)),  # Input layer specifying the input shape
    tf.keras.layers.Dense(64, activation= 'relu'), 
    tf.keras.layers.Dense(128, activation= 'relu'), 
    tf.keras.layers.Dense(num_states, activation= 'softmax')
])
                                
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [18]:
# Train the model
model.fit(X_train, y_train, epochs = 10, validation_data=(X_val, y_val))

Epoch 1/10
1317/1317 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.6931 - loss: 0.9684 - val_accuracy: 0.9451 - val_loss: 0.1996
Epoch 2/10
1317/1317 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9538 - loss: 0.1703 - val_accuracy: 0.9660 - val_loss: 0.1360
Epoch 3/10
1317/1317 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9707 - loss: 0.1217 - val_accuracy: 0.9662 - val_loss: 0.1273
Epoch 4/10
1317/1317 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9768 - loss: 0.0967 - val_accuracy: 0.9776 - val_loss: 0.0895
Epoch 5/10
1317/1317 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9789 - loss: 0.0851 - val_accuracy: 0.9783 - val_loss: 0.0838
Epoch 6/10
1317/1317 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9799 - loss: 0.0794 - val_accuracy: 0.9787 - val_loss: 0.0829
Epoch 7/10
1317/1317 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9834 - loss: 0.0667 - val_accuracy: 0.9808 - val_loss: 0.0735
Epoch 8/10
1317/1317 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9841 - loss: 0.0624 - 

In [19]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9845 - loss: 0.0582
Test Loss: 0.06610069423913956
Test Accuracy: 0.9842355251312256


In [20]:
# output the model
model.save('../model/model2.keras')